# Final Project

#### The Plan
You're planning an evening out on the town with friends. You would rather your activities be nearby each other so you only have to park/take the bus/subway once, then walk to each location. You need to find a cluster of bars/restaurants/activities to do in one area with a set distance. You could use Google and attempt to make your own path, but wouldn't it be easier if you could just pick a starting point, say the activities you'd like to do, and a max walking distance, then get a map with all the points on it?

We will use the Foursquare data to give a demonstration of this for the Tulsa, Oklahoma area (limiting the data size that we pull for this example). We will use the venue category to determine the activities, the latitude and longitude to determine distance, and  